In [20]:
# Lab 12 Character Sequence RNN
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)  # reproducibility
tf.reset_default_graph()

In [21]:
sample = " if you want you"
idx2char = list(set(sample))  # index -> char
char2idx = {c: i for i, c in enumerate(idx2char)}  # char -> idex

In [22]:
# hyper parameters
dic_size = len(char2idx)  # RNN input size (one hot size)
hidden_size = len(char2idx)  # RNN output size
num_classes = len(char2idx)  # final output size (RNN or softmax, etc.)
batch_size = 1  # one sample data, one batch
sequence_length = len(sample) - 1  # number of lstm rollings (unit #)
learning_rate = 0.1


In [23]:
sample_idx = [char2idx[c] for c in sample]  # char to index
x_data = [sample_idx[:-1]]  # X data sample (0 ~ n-1) hello: hell
y_data = [sample_idx[1:]]   # Y label sample (1 ~ n) hello: ello


In [24]:
X = tf.placeholder(tf.int32, [None, sequence_length])  # X data
Y = tf.placeholder(tf.int32, [None, sequence_length])  # Y label


In [25]:
x_one_hot = tf.one_hot(X, num_classes)  # one hot: 1 -> 0 1 0 0 0 0 0 0 0 0

'''
# state_is_tuple : 이전 state를 tuple로 저장
cell = tf.contrib.rnn.BasicLSTMCell(
    num_units=hidden_size, state_is_tuple=True)
'''
cell = tf.contrib.rnn.BasicLSTMCell(
     num_units=hidden_size)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(
    cell, x_one_hot, initial_state=initial_state, dtype=tf.float32)


In [15]:
# FC layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)

In [16]:
# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])


In [17]:
weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)


In [18]:
prediction = tf.argmax(outputs, axis=2)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(50):
        l, _ = sess.run([loss, train], feed_dict={X: x_data, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_data})

        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]

        print(i, "loss:", l, "Prediction:", ''.join(result_str))

0 loss: 2.31705 Prediction: yyyy u   y  y u
1 loss: 2.19862 Prediction: y  yyu yyyyyyyu
2 loss: 2.06886 Prediction: y  you yyy  you
3 loss: 1.88859 Prediction: y   ou      you
4 loss: 1.64126 Prediction: y   ou  an  you
5 loss: 1.31646 Prediction: yu you  ann you
6 loss: 1.00942 Prediction: yu you  ant you
7 loss: 0.739977 Prediction: yf you want you
8 loss: 0.525457 Prediction: yf you want you
9 loss: 0.367246 Prediction: yf you want you
10 loss: 0.251243 Prediction: if you want you
11 loss: 0.170128 Prediction: if you want you
12 loss: 0.115286 Prediction: if you want you
13 loss: 0.0768127 Prediction: if you want you
14 loss: 0.0506168 Prediction: if you want you
15 loss: 0.0333266 Prediction: if you want you
16 loss: 0.022125 Prediction: if you want you
17 loss: 0.0149369 Prediction: if you want you
18 loss: 0.0103241 Prediction: if you want you
19 loss: 0.00733763 Prediction: if you want you
20 loss: 0.0053726 Prediction: if you want you
21 loss: 0.00405191 Prediction: if you want